In [2]:
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.35.2 peft==0.4.0 accelerate==0.21.0
%pip install -q -U datasets py7zr einops tensorboardX
!pip install evaluate
!pip install bert_score
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip ins

In [3]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
results

{'precision': [1.0, 0.9999999403953552],
 'recall': [1.0, 0.9999999403953552],
 'f1': [1.0, 0.9999999403953552],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.2)'}

In [8]:
import pandas as pd
gsm8k = pd.read_csv('data/gsm8k_questions.csv')

In [ ]:
def score(df1, df2, df1var, df2var):
    scores = []
    for i in range(0, len(df1)):
        for j in range(0, len(df2)):
            ref = df1.iloc[i][df1var]
            ref = str(ref)
            pred = df2.iloc[j][df2var]
            pred = str(pred)
            results = bertscore.compute(predictions=[pred], references=[ref], lang="en")
            scores.append(results)
    return scores
scores = score(gsm8k, gsm8k, 'instruction', 'instruction')

In [20]:
def concatenate(df, varname):
    questions = []
    for i in range(0, len(df)):
        question = df.iloc[i][varname]
        questions.append(question)
    return questions
gsm8k_questions = concatenate(gsm8k, 'instruction')
scores = []
for i in range(0, len(gsm8k)):
    ref = gsm8k.iloc[i]['instruction']
    ref = [str(ref)]
    ref *= len(gsm8k_questions)
    score = bertscore.compute(predictions=gsm8k_questions, references=ref, lang="en")
    scores.append(score)

KeyboardInterrupt: 